In [1]:
# clean_zillow_city_separate.py
import pandas as pd
from pathlib import Path

# -------- CONFIG --------
STATE_ABBR = "NC"
START_DATE = "2015-01-01"

# Point these to your raw files
FILES = {
    "ZHVI":          "data/raw/ZHVI.csv",
    "ZORI":          "data/raw/ZORI.csv",
    "Inventory":     "data/raw/Inventory.csv",
    "DaysToPending": "data/raw/DaysToPending.csv",
    # "PriceCuts":   "data/raw/PriceCuts.csv",  # add when you have it
}

# Missing-data policy per metric
POLICY = {
    # prices/rents: no forward-fill, small internal interpolation only
    "ZHVI":          dict(drop_nonpositive=True,  ffill_limit=0, interpolate_limit=3),
    "ZORI":          dict(drop_nonpositive=True,  ffill_limit=0, interpolate_limit=3),
    # reporting-lag metrics: allow 1-mo ffill, tiny interpolation
    "Inventory":     dict(drop_nonpositive=False, ffill_limit=1, interpolate_limit=2),
    "DaysToPending": dict(drop_nonpositive=False, ffill_limit=1, interpolate_limit=2),
    # "PriceCuts":   dict(drop_nonpositive=False, ffill_limit=0, interpolate_limit=2),
}

OUT_DIR = Path("data/interim")  # per-metric outputs go here
OUT_DIR.mkdir(parents=True, exist_ok=True)

ID_COLS = ["RegionID","SizeRank","RegionName","RegionType","StateName","State","Metro","CountyName"]

# -------- HELPERS --------
def detect_date_columns(df: pd.DataFrame) -> list[str]:
    """Columns whose header can be parsed as a date (any common format)."""
    cols = []
    for c in df.columns:
        if c in ID_COLS: 
            continue
        if not any(ch.isdigit() for ch in c):
            continue
        dt = pd.to_datetime(c, errors="coerce")
        if pd.isna(dt):
            dt = pd.to_datetime(c, errors="coerce", dayfirst=True)
        if not pd.isna(dt):
            cols.append(c)
    return cols

def state_mask(df: pd.DataFrame, abbr: str) -> pd.Series:
    if "State" in df.columns:
        return df["State"].astype(str).str.upper().eq(abbr)
    if "StateName" in df.columns:
        s = df["StateName"].astype(str).str.upper()
        return s.eq(abbr) | s.eq("NORTH CAROLINA")
    raise ValueError("No State/StateName column found.")

def fill_small_gaps(g: pd.DataFrame, metric: str, ffill_limit: int, interpolate_limit: int) -> pd.DataFrame:
    g = g.sort_values("Date").copy()
    if ffill_limit > 0:
        g[metric] = g[metric].ffill(limit=ffill_limit)  # handle 1-month reporting lag
    g[metric] = g[metric].interpolate(
        method="linear",
        limit=interpolate_limit,        # fill small internal runs only
        limit_direction="both",
        limit_area="inside"
    )
    return g

def clean_zillow_city_file(
    file_path: str,
    metric_name: str,
    state_abbr: str = STATE_ABBR,
    start_date: str = START_DATE,
    drop_nonpositive: bool = True,
    ffill_limit: int = 0,
    interpolate_limit: int = 3,
) -> pd.DataFrame:
    print(f"\n📥 {metric_name}: reading {file_path}")
    try:
        df = pd.read_csv(file_path, encoding="utf-8")
    except UnicodeDecodeError:
        df = pd.read_csv(file_path, encoding="latin-1")
    print("Raw:", df.shape)

    # Tidy headers / strings
    df.columns = df.columns.str.strip()
    obj = df.select_dtypes(include="object").columns
    df[obj] = df[obj].apply(lambda s: s.astype(str).str.strip())

    # Detect date headers and melt to long
    date_cols = detect_date_columns(df)
    if not date_cols:
        raise ValueError("No date-like columns detected.")
    print(f"Date columns: {len(date_cols)}")
    id_cols = [c for c in ID_COLS if c in df.columns]

    long = df.melt(id_vars=id_cols, value_vars=date_cols,
                   var_name="DateStr", value_name=metric_name)
    # Parse header dates (any format)
    dt = pd.to_datetime(long["DateStr"], errors="coerce")
    dt_na = dt.isna()
    if dt_na.any():
        dt.loc[dt_na] = pd.to_datetime(long.loc[dt_na, "DateStr"], errors="coerce", dayfirst=True)
    long["Date"] = dt
    long = long.drop(columns=["DateStr"]).dropna(subset=["Date"])

    # Metric to numeric; treat <=0 as missing for price/rent
    long[metric_name] = pd.to_numeric(long[metric_name], errors="coerce")
    if drop_nonpositive:
        long.loc[long[metric_name] <= 0, metric_name] = pd.NA

    # Filters
    long = long[state_mask(long, state_abbr)]
    long = long[long["Date"] >= pd.Timestamp(start_date)]

    # De-dup, then fill small gaps per city
    long = (long.sort_values(["RegionID","Date"])
                .drop_duplicates(subset=["RegionID","Date"], keep="last")
                .groupby("RegionID", group_keys=False)
                .apply(lambda g: fill_small_gaps(g, metric_name, ffill_limit, interpolate_limit))
                .reset_index(drop=True))

    # Final tidy columns
    keep = [c for c in ID_COLS + ["Date", metric_name] if c in long.columns]
    long = long[keep].sort_values(["RegionName","Date"]).reset_index(drop=True)

    # Sanity
    dups = long.duplicated(subset=["RegionID","Date"]).sum()
    print(f"Final {metric_name}: {long.shape} | Duplicates (RegionID+Date): {dups}")
    return long

# -------- RUN: PER-METRIC OUTPUTS ONLY --------
if __name__ == "__main__":
    for metric, path in FILES.items():
        p = POLICY.get(metric, dict(drop_nonpositive=False, ffill_limit=0, interpolate_limit=2))
        df_clean = clean_zillow_city_file(
            file_path=path,
            metric_name=metric,
            drop_nonpositive=p["drop_nonpositive"],
            ffill_limit=p["ffill_limit"],
            interpolate_limit=p["interpolate_limit"],
        )
        out_path = OUT_DIR / f"nc_city_{metric}_2015present.csv"
        df_clean.to_csv(out_path, index=False, encoding="utf-8")
        print(f"✅ Saved: {out_path}")



📥 ZHVI: reading data/raw/ZHVI.csv
Raw: (21489, 314)
Date columns: 306


C:\Users\Shalin\AppData\Local\Temp\ipykernel_53888\590637863.py:121: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_small_gaps(g, metric_name, ffill_limit, interpolate_limit))


Final ZHVI: (84294, 10) | Duplicates (RegionID+Date): 0
✅ Saved: data\interim\nc_city_ZHVI_2015present.csv

📥 ZORI: reading data/raw/ZORI.csv
Raw: (3768, 134)
Date columns: 126


C:\Users\Shalin\AppData\Local\Temp\ipykernel_53888\590637863.py:121: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_small_gaps(g, metric_name, ffill_limit, interpolate_limit))
C:\Users\Shalin\AppData\Local\Temp\ipykernel_53888\590637863.py:121: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_small_gaps(g, metric_name, ffill_limit, interpolate_limit))


Final ZORI: (15120, 10) | Duplicates (RegionID+Date): 0
✅ Saved: data\interim\nc_city_ZORI_2015present.csv

📥 Inventory: reading data/raw/Inventory.csv
Raw: (928, 93)
Date columns: 88
Final Inventory: (3256, 7) | Duplicates (RegionID+Date): 0
✅ Saved: data\interim\nc_city_Inventory_2015present.csv

📥 DaysToPending: reading data/raw/DaysToPending.csv
Raw: (756, 93)
Date columns: 88
Final DaysToPending: (2640, 7) | Duplicates (RegionID+Date): 0
✅ Saved: data\interim\nc_city_DaysToPending_2015present.csv


C:\Users\Shalin\AppData\Local\Temp\ipykernel_53888\590637863.py:121: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(lambda g: fill_small_gaps(g, metric_name, ffill_limit, interpolate_limit))


In [2]:
import pandas as pd, csv, re
from pathlib import Path

def tableau_proof(in_path, out_path):
    # 1) read robustly, strip nulls
    raw = Path(in_path).read_text(encoding="utf-8", errors="replace").replace("\x00", "")
    Path(in_path).write_text(raw, encoding="utf-8")

    df = pd.read_csv(in_path, engine="python")  # lets pandas sniff delimiter

    # 2) tidy headers/strings, drop unnamed
    df.columns = [re.sub(r"\s+", " ", c).strip() for c in df.columns]
    df = df.loc[:, ~df.columns.str.contains(r"^Unnamed")]

    # 3) normalize Date to ISO
    if "Date" in df.columns:
        df["Date"] = pd.to_datetime(df["Date"], errors="coerce")
        df = df.dropna(subset=["Date"])
        df["Date"] = df["Date"].dt.strftime("%Y-%m-%d")

    # 4) force numeric where possible (without changing text cols)
    id_like = {"RegionID","SizeRank","RegionName","RegionType","StateName","State","Metro","CountyName","Date"}
    for c in df.columns:
        if c not in id_like:
            df[c] = pd.to_numeric(df[c], errors="ignore")

    # 5) write CSV with BOM + CRLF + quote-all (Tableau-safe)
    Path(out_path).parent.mkdir(parents=True, exist_ok=True)
    df.to_csv(
        out_path,
        index=False,
        encoding="utf-8-sig",        # BOM
        lineterminator="\r\n",       # Windows line endings
        quoting=csv.QUOTE_ALL        # commas inside fields are safe
    )
    print(f"Saved: {out_path}  {df.shape}")

# Example: point to your cleaned files
tableau_proof("data/interim/nc_city_ZHVI_2015present.csv",          "data/interim/tableau/nc_city_ZHVI_2015present.csv")
tableau_proof("data/interim/nc_city_ZORI_2015present.csv",          "data/interim/tableau/nc_city_ZORI_2015present.csv")
tableau_proof("data/interim/nc_city_Inventory_2015present.csv",     "data/interim/tableau/nc_city_Inventory_2015present.csv")
tableau_proof("data/interim/nc_city_DaysToPending_2015present.csv", "data/interim/tableau/nc_city_DaysToPending_2015present.csv")


C:\Users\Shalin\AppData\Local\Temp\ipykernel_53888\2309668832.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")


Saved: data/interim/tableau/nc_city_ZHVI_2015present.csv  (84294, 10)
Saved: data/interim/tableau/nc_city_ZORI_2015present.csv  (15120, 10)
Saved: data/interim/tableau/nc_city_Inventory_2015present.csv  (3256, 7)
Saved: data/interim/tableau/nc_city_DaysToPending_2015present.csv  (2640, 7)


C:\Users\Shalin\AppData\Local\Temp\ipykernel_53888\2309668832.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
C:\Users\Shalin\AppData\Local\Temp\ipykernel_53888\2309668832.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
C:\Users\Shalin\AppData\Local\Temp\ipykernel_53888\2309668832.py:25: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df[c] = pd.to_numeric(df[c], errors="ignore")
